In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np

from bokeh.plotting import figure, show, save
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.io import export_png

import shapely

import sys

sys.path.append("../")

from Code.helper_functions import initial_loading
from Code.loading_data import load_api_data

### Errors on loading API data
It appeared that there were some problems regarding the loading of the data using the API. The connection got actively refused. After some testing a backup plan using a try-except clausule was implemented to prevent this from happening again

In [ ]:
load_api_data(prnt=True)

In [ ]:
df_api = pd.read_csv('../Data/afval_cluster.csv', delimiter=';')
df_api = df_api[df_api['cluster_datum_einde_cluster'].isna()]
df_api = df_api[['cluster_geometrie', 'cluster_fractie_aantal', 'cluster_fractie_volume', 'bag_adres_openbare_ruimte_naam', 'gbd_buurt_code']]
df_api['cluster_x'] = df_api['cluster_geometrie'].apply(lambda x: x.split('(')[1].split(' ')[0])
df_api['cluster_y'] = df_api['cluster_geometrie'].apply(lambda x: x.split()[1][:-1])
df_api = df_api.drop(['cluster_geometrie'], axis=1).rename(columns={'cluster_fractie_aantal':'aantal_per_fractie', 'cluster_fractie_volume':'volume_per_fractie', 'bag_adres_openbare_ruimte_naam':'street_name', 'gbd_buurt_code':'buurt'})

### Reassambling the score and penalty function
There was some bad coding going on in terms of the score and penalty function. There was also new reassignment of the normalization factor. An optional parameter to print all factors instead of the total penalties was also added

In [ ]:
df_zo = pd.read_csv('../Results/20200510 - cluster optimization/hillclimber_best_config20200509-1228.csv')
all_households, rel_poi_df, joined, df_afstandn2 = initial_loading()

Do you want to use addresses instead of clusters?True
What stadsdeel do you want to make as a subsection(optional parameter)?
What is the maximum amount of containers in a cluster that is considered to be useful?8
Where to get db files(local/online)?local
DB relation POIs loaded


..\Code\loading_data.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  verblijfsobjecten['bag'] = verblijfsobjecten['split'].apply(lambda x: x[3])\
..\Code\loading_data.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ligtin_bag_pnd_identificatie'] = \
..\Code\loading_data.py:369: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

distance matrix loaded
API data loaded
Table all households created
API and DB joined
containers per cluster determined


In [ ]:
from Code.helper_functions import add_shortest_distances_to_all_households, analyze_candidate_solution
from Code.loading_data import create_aansluitingen

joined_cluster_distance = joined.set_index('s1_afv_nodes')\
    .join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index()\
    .rename(columns={'index': 'van_s1_afv_nodes'})

good_result = \
    add_shortest_distances_to_all_households(all_households,
                                             joined_cluster_distance,
                                             use_count=True)

aansluitingen = create_aansluitingen(good_result, joined_cluster_distance, use_count=True)

joined_cluster_distance, good_result_rich, aansluitingen,avg_distance, penalties = analyze_candidate_solution(joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True, return_all=True)

### Optimization of other fractions in places where garbage is collected from the curbs
As nearly half of the city uses curbstone rest garbage collection, these can't just be excluded. Moreover, it also prevents dumping of undeseriable containers to these areas. The optional parameter clean has to be set to False for use. Its impact on the score function is to be determined further

In [ ]:
temp = good_result[good_result['uses_container'] == False]
# temp['rest_afstand'] = 9
# temp['poi_rest'] = 9
# good_result[good_result['uses_container']== False]['rest_afstand'] = 10
# good_result[good_result['uses_container'] == False]['poi_rest'] = 10
# good_result = good_result[good_result['uses_container'] == True]
# good_result = good_result.append(temp)
good_result.loc[~good_result['uses_container'], 'rest_afstand'] = np.nan
good_result.loc[~good_result['uses_container'], 'poi_rest'] = np.nan
good_result

In [ ]:
joined_cluster_distance = joined.set_index('s1_afv_nodes')\
    .join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index()\
    .rename(columns={'index': 'van_s1_afv_nodes'})

good_result = \
    add_shortest_distances_to_all_households(all_households,
                                             joined_cluster_distance,
                                             use_count=True)

aansluitingen = create_aansluitingen(good_result, joined_cluster_distance, use_count=True)

joined_cluster_distance, good_result_rich, aansluitingen,avg_distance, penalties = analyze_candidate_solution(joined, all_households, rel_poi_df, df_afstandn2, clean=False, use_count=True, return_all=True)

### Plotting of distances and penalties
As optimization of just one of the parameters is possible within reasonable limits, the search arises for a way to compare these. One of the mentioned options was to optimize on one parameter, and hopefully find some point that is ideal. As the normal distance doesn't show anything, the best distance should be extracted

In [ ]:
def plot_optimization(filename):
    df_plot1 = pd.read_csv(filename)
    df_plot1['diff'] = df_plot1['penalties'] - df_plot1['best']
    # df_plot1['total'] = df_plot1['avg_distance'] + df_plot1['penalties']
    df_plot1['best_dist'] = df_plot1[df_plot1['diff'] < 0]['avg_distance']
    df_plot1['best_dist'] = df_plot1['best_dist'].fillna(method='ffill')
    ax = df_plot1['best_dist'].plot()
    ax = df_plot1['best'].plot(secondary_y=True)
    return ax
    # ax = df_plot1['total'].plot(secondary_y=True)

In [ ]:
fig1 = plot_optimization('../Results/20200510 - cluster optimization/hillclimber20200509-1228.csv')
fig1.set_title('Penalty-based optimization of Zuid-Oost with display of best average distance')

In [ ]:
plot_optimization('../Results/20200510 - cluster optimization/hillclimber20200509-2116.csv')

In [ ]:
plot_optimization('../Results/20200510 - cluster optimization/hillclimber20200510-1050.csv')

In [ ]:
plot_optimization('../Results/20200510 - cluster optimization/hillclimber20200512-0813.csv')

It appears that an optimization approach where one of the parameters is chosen and constraints are set on the detoriation of the other may be an appropriate way to go. 

### Weight factors based on the amount of containers per fraction
The summation of the penalties and walking distances for the different fractions is hard to make, as some kind of weighing needs to be applied. One of the new approaches would be to determine these weights based on the amount of containers that is present, as this would probably correlate with the amount of garbage produced and the relative importance of the different fractions.

In [ ]:
joined_clean = joined[joined['totaal'] < 9]
temp = joined_clean[['rest', 'plastic', 'papier', 'glas', 'textiel', 'totaal']].sum()
temp

Based on these numbers the weights would be assigned the following way: Rest(0.61458), Plastic(0.088776), Papier(0.15774), Glas(0.11350) and Textiel(0.025397). This will be set as the default value for the penalties.

### Preventing the extra placement of Rest containers in light grey areas
As there are certain areas where restafval is picked up, it doesn't make sense to place extra rest containers there. It should be set as a hard constraint that this is not possible, to prevent certain dumping of containers in these areas. The first snippet of code is added to the loading phases to keep track of the possibilities for each cluster to add rest containers. The constraint that move_rest has to be true is added to the optimization algorithms

In [ ]:
from Code.loading_data import load_geodata_containers, address_in_service_area
shapefile = load_geodata_containers()
joined['move_rest'] = joined.apply(lambda row: address_in_service_area(row['cluster_x'], row['cluster_y'], polygon_list=shapefile), axis=1)
# joined[~joined['move_rest']][['rest', 'plastic', 'papier', 'glas', 'textiel', 'totaal']].sum()

In [ ]:
import random
r = joined
mod_max = 5

fractions = ['rest', 'plastic', 'papier', 'glas', 'textiel']
no_modifications = random.randint(1, mod_max)
#         print(no_modifications)
for j in range(no_modifications):
    valid = False
    while not valid:
        location_a = random.randint(0, r.shape[0]-1)
        fraction_a = random.choice(fractions)
        location_b = random.randint(0, r.shape[0]-1)
        fraction_b = random.choice(fractions)
        print(location_a, fraction_a, location_b, fraction_b, r.at[location_a, 'move_rest'])

        if int(r.at[location_a, fraction_b]) > 0 and \
                int(r.at[location_b, fraction_a]) > 0 and \
                fraction_a != fraction_b and not (fraction_a == 'rest' and
                                                  ~r.at[location_a, 'move_rest']):

            r.at[location_a, fraction_a] = \
                int(r.at[location_a, fraction_a]) + 1
            r.at[location_a, fraction_b] = \
                int(r.at[location_a, fraction_b]) - 1
            r.at[location_b, fraction_a] = \
                int(r.at[location_b, fraction_a]) - 1
            r.at[location_b, fraction_b] = \
                int(r.at[location_b, fraction_b]) + 1

            valid = True